<center>
<h1 id=""><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"></h1>
<h1 id="unSkript-Runbooks">unSkript Runbooks</h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong></h3>
<strong>Rollback k8s Deployment and Update Jira using unSkript actions.</strong></div>
</center><center>
<h2 id="Rollback-k8s-Deployment-and-Update-Jira">Rollback k8s Deployment and Update Jira</h2>
</center>
<h1 id="Steps-Overview"><sub>Steps Overview</sub></h1>
<ol>
<li>&nbsp;Get the Deployment Rollout status</li>
<li>&nbsp;Kubectl rollout deployment</li>
<li>&nbsp;Change JIRA Issue Status</li>
</ol>

<h3 id="Get-All-The-Namespaces">Get All The Namespaces</h3>
<p>In this action, we collect all the namespaces available in the cluster as a list. This action only executes if the namespace parameter is not given.</p>
<ul>
<li><strong>Input parameters:</strong>&nbsp; <code>kubectl_command</code></li>
<li><strong>Output variable:</strong>&nbsp; <code>namespace_list</code></li>
</ul>

In [15]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_command_printer(output):
    if output is None:
        return
    print(output)


@beartype
def k8s_kubectl_command(handle, kubectl_command: str) -> str:
    """k8s_kubectl_command executes the given kubectl command on the pod

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type kubectl_command: str
        :param kubectl_command: The Actual kubectl command, like kubectl get ns, etc..

        :rtype: String, Output of the command in python string format or Empty String in case of Error.
    """
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return str()

    result = handle.run_native_cmd(kubectl_command)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        print(
            f"Error while executing command ({kubectl_command}): {result.stderr}")
        return str()

    return result.stdout


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "kubectl_command": "\\"kubectl get namespaces -o json\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not namespace",
    "condition_result": true
    }''')
task.configure(outputName="namespace_list")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_command, lego_printer=k8s_kubectl_command_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Modify Output</h3>
<p>In this action, we modify the output which collects from the Gathering Information cell and returns a list of all the namespaces or given namespaces.</p>
<ul>
<li><strong>Output variable:</strong>&nbsp; <code>namespace_data</code></li>
</ul>

In [16]:
import json

namespace_data = []
try:
    if namespace_list:
        namespace_details = json.loads(namespace_list)
        for i in namespace_details["items"]:
            namespace_data.append(i["metadata"]["name"])
except Exception as e:
    namespace_data.append(namespace)

<h3 id="Get-Deployment-Rollout-Status">Get Deployment Rollout Status</h3>
<p>Here we will use the unSkript&nbsp;<strong>Get Deployment Rollout Status</strong> action. This action is used to identify the status of deployment for the namespace and return a list of a dictionary that contains the deployments which failed.</p>
<ul>
<li><strong>Input parameters:</strong>&nbsp; <code>namespace, deployment</code></li>
<li><strong>Output variable:</strong>&nbsp; <code>deployment_data</code></li>
</ul>

In [25]:
#
# Copyright (c) 2022 unSkript.com
# All rights reserved.
#

from pydantic import BaseModel, Field
from typing import Optional
from unskript.legos.utils import CheckOutput, CheckOutputStatus
import pprint
import json

from beartype import beartype
@beartype
def k8s_get_deployment_rollout_status_printer(output):
    if output is None:
        return
    if isinstance(output, CheckOutput):
        print(output.json())
    else:
        pprint.pprint(output)


@beartype
def k8s_get_deployment_rollout_status(handle, deployment: str = "", namespace: str = "") -> CheckOutput:
    """k8s_get_deployment_rollout_status executes the command and give failed deployment list

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type deployment: str
        :param deployment: Deployment Name.

        :type namespace: str
        :param namespace: Kubernetes Namespace.

        :rtype: CheckOutput with status result and list of failed deployments.
    """
    result = []
    if handle.client_side_validation != True:
        print(f"K8S Connector is invalid: {handle}")
        return CheckOutput(status=CheckOutputStatus.RUN_EXCEPTION,
                               objects=[],
                               error=str())

    status_details = ""
    if namespace and deployment:
        name_cmd = "kubectl get deployment " + deployment + " -n " + namespace + " -o json"
        exec_cmd = handle.run_native_cmd(name_cmd)
        status_op = exec_cmd.stdout
        status_details = json.loads(status_op)

    if not namespace and not deployment:
        name_cmd = "kubectl get deployments --all-namespaces -o json"
        exec_cmd = handle.run_native_cmd(name_cmd)
        status_op = exec_cmd.stdout
        status_details = json.loads(status_op)

    if namespace and not deployment:
        name_cmd = "kubectl get deployment -n " + namespace + " -o json"
        exec_cmd = handle.run_native_cmd(name_cmd)
        status_op = exec_cmd.stdout
        status_details = json.loads(status_op)

    if deployment and not namespace:
        name_cmd = "kubectl get deployment " + deployment + " -o json"
        exec_cmd = handle.run_native_cmd(name_cmd)
        status_op = exec_cmd.stdout
        status_details = json.loads(status_op)

    if status_details:
        if "items" in status_details:
            for items in status_details["items"]:
                namespace_name = items["metadata"]["namespace"]
                deployment_name = items["metadata"]["name"]
                replica_details = items["status"]["conditions"]
                for i in replica_details:
                    deployment_dict = {}
                    if "FailedCreate" in i["reason"] and "ReplicaFailure" in i["type"] and "True" in i["status"]:
                        deployment_dict["namespace"] = namespace_name
                        deployment_dict["deployment_name"] = deployment_name
                        result.append(deployment_dict)
                    if "ProgressDeadlineExceeded" in i["reason"] and "Progressing" in i["type"] and "False" in i["status"]:
                        deployment_dict["namespace"] = namespace_name
                        deployment_dict["deployment_name"] = deployment_name
                        result.append(deployment_dict)
        else:
            namespace_name = status_details["metadata"]["namespace"]
            deployment_name = status_details["metadata"]["name"]
            replica_details = status_details["status"]["conditions"]
            for i in replica_details:
                deployment_dict = {}
                if "FailedCreate" in i["reason"] and "ReplicaFailure" in i["type"] and "True" in i["status"]:
                    deployment_dict["namespace"] = namespace_name
                    deployment_dict["deployment_name"] = deployment_name
                    result.append(deployment_dict)
                if "ProgressDeadlineExceeded" in i["reason"] and "Progressing" in i["type"] and "False" in i["status"]:
                    deployment_dict["namespace"] = namespace_name
                    deployment_dict["deployment_name"] = deployment_name
                    result.append(deployment_dict)

    if len(result) != 0:
        return CheckOutput(status=CheckOutputStatus.FAILED,
                   objects=result,
                   error=str(""))
    else:
        return CheckOutput(status=CheckOutputStatus.SUCCESS,
                   objects=result,
                   error=str(""))



task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "namespace": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "namespace_data",
    "iter_parameter": "namespace"
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(namespace_data)>0",
    "condition_result": true
    }''')
task.configure(outputName="deployment_data")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_get_deployment_rollout_status, lego_printer=k8s_get_deployment_rollout_status_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Modify Output</h3>
<p>In this action, we modify the output which collects from step 1 and return a list of dictionaries for the failed deployments.</p>
<ul>
<li><strong>Output variable:</strong>&nbsp; <code>rollout_deployment</code></li>
</ul>

In [24]:
rollout_deployment = []
if deployment_data:
    for k, v in deployment_data.items():
        get_json = json.loads(v.json())
        if get_json["status"] == 2:
            for deployments in vol["objects"]:
                rollout_deployment.append(deployments)

<h3 id="Kubectl-rollout-deployment">Kubectl rollout deployment</h3>
<p>Here we will use the unSkript <strong>Kubectl rollout deployment</strong> action. This action is used to roll back the deployment to a stable version.</p>
<ul>
<li><strong>Input parameters:</strong>&nbsp; <code>k8s_cli_string, deployment,&nbsp;namespace</code></li>
<li><strong>Output variable:</strong>&nbsp; <code>rollback_status</code></li>
</ul>

In [ ]:
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def k8s_kubectl_rollout_deployment_printer(data: str):
    if data is None:
        print("Error while executing command")
        return

    print (data)

@beartype
def k8s_kubectl_rollout_deployment(handle, k8s_cli_string: str, deployment: str, namespace: str) -> str:
    k8s_cli_string = k8s_cli_string.format(deployment, namespace)
    result = handle.run_native_cmd(k8s_cli_string)
    if result is None or hasattr(result, "stderr") is False or result.stderr is None:
        return None
    return result.stdout


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "deployment": "iter.get(\\"deployment_name\\")",
    "k8s_cli_string": "kubectl rollout undo deploy {deployment} -n {namespace}",
    "namespace": "iter.get(\\"namespace\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "deployment_data",
    "iter_parameter": ["deployment","namespace"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(rollout_deployment)>0",
    "condition_result": true
    }''')
task.configure(outputName="rollback_status")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_kubectl_rollout_deployment, lego_printer=k8s_kubectl_rollout_deployment_printer, hdl=hdl, args=args)

<h3 id="Change-JIRA-Issue-Status">Change JIRA Issue Status</h3>
<p>Here we will use unSkript <strong>Change JIRA Issue Status</strong> action. This action is used to update the status of the Jira issue. It will only execute if the issue id is given.</p>
<ul>
<li><strong>Input parameters:</strong>&nbsp; <code>issue_id, status, transition</code></li>
<li><strong>Output variable:</strong>&nbsp; <code>update_status</code></li>
</ul>

In [26]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

from jira.client import JIRA
from pydantic import BaseModel, Field
from typing import Optional
import pprint

pp = pprint.PrettyPrinter(indent=4)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        print('\n')
        return output
    return Printer


@legoPrinter
@beartype
def jira_issue_change_status(hdl: JIRA, issue_id: str, status: str, transition: str = ""):
    """jira_get_issue_status gets the status of a given Jira issue.
        :type issue_id: str
        :param issue_id: ID of the issue whose status we want to fetch (eg ENG-14)
        :rtype: String with issue status fetched from JIRA API
    """

    # Input param validation.
    issue = hdl.issue(issue_id)

    if transition:
        transitions = hdl.transitions(issue)
        t = [t for t in transitions if t.get('name') == status]

        if len(t) > 1:
            print("Multiple transitions possible for JIRA issue. Please select transition number to use", [
                t.get('id') for t in transitions if t.get('name') == status])
            return
        else:
            transition = t[0].get('id')

    hdl.transition_issue(issue, transition)
    return


def unskript_default_printer(output):
    if isinstance(output, (list, tuple)):
        for item in output:
            print(f'item: {item}')
    elif isinstance(output, dict):
        for item in output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(output)

task = Task(Workflow())
task.configure(printOutput=True)
task.configure(inputParamsJson='''{
    "issue_id": "issue_id",
    "status": "\\"Done\\""
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "issue_id",
    "condition_result": true
    }''')
task.configure(outputName="update_status")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(jira_issue_change_status, lego_printer=unskript_default_printer, hdl=hdl, args=args)

### Conclusion
<p>In this Runbook, we demonstrated the use of unSkript's AWS and Jira actions to roll back the Kubernetes deployment to the previous stable version and update the issue status in jira. To view the full platform capabilities of unSkript please visit <a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>